# Perform hyperparameter tuning of the model.

In [ ]:
import os
import json
from functools import partial
import optuna

## Setup script params

Only modify below cell and run rest of the notebook. 

Alternatively, if you have `papermill` installed, you can run the notebook with:

`papermill 99-hyperparameters_tuning.ipynb out_notebook.ipynb -f hyperparameters_tuning_config.yml`

from the command line, where `hyperparameters_tuning_config.yml` stores the parameters below.

In [ ]:
# Outputs directory
OUTPUTS_DIR = 'hyperparam_tuning_data/hyperparam_tuning_output'

# Path to hyperparameters grid in JSON format
HYPERPARAMETERS_GRID_FILEPATH = 'hyperparam_tuning_data/example_hyperparameters_grid.json'

# Here, setup fixed hyperparams for your training-evaluation procedure - single Optuna trial. Example hyperparams below
NUM_FOLDS = 5
NUM_EPOCHS = 10
BATCH_SIZE = 4
CROSS_VAL_VAL_SIZE = 0.2
NUM_WORKERS = 1
ACCELERATOR = "cpu"   # "gpu" or "tpu"

# Setup hyperparams corresponding to actual model's hyperparameters search
# Hyperparams for hyperparameter tuning
OPTUNA_METRIC_DIRECTION = 'minimize'   # 'minimize' or 'maximize'
OPTUNA_N_TRIALS = 2   # Number of trials to run
OPTUNA_N_JOBS = 1   # Number of parallel jobs
OPTUNA_SEED = 42

## Utils

In [ ]:
def get_hparam_search_space(trial, hyperparameters_grid):
    """
    Generates a hyperparameter search space for a given trial.

    Args:
        trial (optuna.trial.Trial): The trial for which to generate the search space.
        hyperparameters_grid (dict): A dictionary defining the hyperparameters and their possible values.

    Returns:
        dict: A dictionary containing the hyperparameters and their suggested values for the trial.
    """
    hyperparams = {}
    for hp_name, hp_params in hyperparameters_grid.items():
        if hp_params['type'] == 'int':
            hyperparams[hp_name] = trial.suggest_int(hp_name, hp_params['low'], hp_params['high'])
        elif hp_params['type'] == 'categorical':
            hyperparams[hp_name] = trial.suggest_categorical(hp_name, hp_params['choices'])
        elif hp_params['type'] == 'float':
            hyperparams[hp_name] = trial.suggest_float(hp_name, hp_params['low'], hp_params['high'])
    return hyperparams

def load_json(file_path):
    """
    Loads a JSON file.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The loaded JSON file.
    """
    with open(file_path, 'r') as f:
        return json.load(f)
    
def save_json(file_path, data):
    """
    Saves a dictionary to a JSON file.

    Args:
        file_path (str): The path to the JSON file.
        data (dict): The dictionary to save.
    """
    with open(file_path, 'w') as f:
        json.dump(data, f)

## Load data

In [ ]:
# Load hyperparameters grid
hyperparameters_grid = load_json(HYPERPARAMETERS_GRID_FILEPATH)

Hyperparameters grid is a dictionary with parameter names as keys and dictionaries as values. For every hyperparameter, a dictionary has to have the following fields:

- type ('int', 'float' or 'categorical')
- if int or float, another fields are:
    - 'low' (int or float): lower threshold on param's values
    - 'high' (int or float): upper threshold on param's values
    - 'step': (int or float): a step fo discretization (optional)
    - 'log' (bool): a flag to sample the value from the log domain or not (optional)

- if 'categorical', another fields are:
    - 'choices' (list): possible parms values

See https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html for reference.

See `get_hparam_search_space` function above to see how this dictionary is converted to Optuna format.

In [ ]:
# View the loaded hyperparameters grid
for k, v in hyperparameters_grid.items():
    print(f'{k}: {v}')

## Code needed for running the trial..

Here, you can establish code and variables which are common for all the trials, e.g. load data, datasets and dataloaders, load model etc. You can also do it later in the actual trial function which is needed by Optuna.

In [ ]:
# Your code here
...

## Optuna objective function

Opuna requires an objective function to be defined, i.e. a function which takes a Trial object as an argument (could be along with some other arguments) and performs a single trial with a combination of hyperparams. The function returns a metric to be optimized. We will create a function which takes in a model and a hyperparameter grid as additional arguments.

In [ ]:
def model_objective(trial, hyperparameters_grid):
    """
    Objective function for Optuna to optimize.

    Args:
        trial (optuna.trial.Trial): The trial for which to generate the search space.

    Returns:
        float: The value of the objective function.
    """
    # Generate hyperparameters for the trial
    hyperparams = get_hparam_search_space(trial, hyperparameters_grid)

    # Example of how to access hyperparameters
    current_learning_rate = hyperparams['learning_rate']

    # Your code here - you need to have some objective value at the end
    ...

    # Example outcome - should be effect of your code above
    objective_value = -1
    example_variable = "test"

    # Example how you can record a user-defined variable fot this trial
    trial.set_user_attr("example_variable", example_variable)   # This is just an example, optuna records the parameters automatically

    return objective_value

## Perform hyperparam tuning

In [ ]:
if not os.path.exists(OUTPUTS_DIR):
    os.makedirs(OUTPUTS_DIR)

# Create actual objective function using partial - pass in the hyperparameters grid
objective_func = partial(model_objective, hyperparameters_grid=hyperparameters_grid)

# Create a study object
study = optuna.create_study(direction=OPTUNA_METRIC_DIRECTION, 
                            sampler=optuna.samplers.TPESampler(seed=OPTUNA_SEED))   # Default sampler is TPESampler, providing seed for reproducibility

# Start the hyperparameter tuning
study.optimize(objective_func, n_trials=OPTUNA_N_TRIALS, n_jobs=OPTUNA_N_JOBS)
study_df = study.trials_dataframe()

# Save study dataframe
study_df.to_csv(os.path.join(OUTPUTS_DIR, "study_results.csv"), index=False)
# Save best params
save_json(os.path.join(OUTPUTS_DIR, "best_params.json"), study.best_params)